# Hyperlane Validator Announce Analysis - Sepolia, Pruv & Arbitrum Sepolia Networks

This notebook analyzes the gas costs and transaction patterns for Hyperlane validator announce transactions on Sepolia, Pruv, and Arbitrum Sepolia networks.

In [1]:
import json
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import display

# Define chains for processing
chains = ['arbitrumsepolia', 'pruv', 'sepolia']

## 1. Load and Parse Data

In [2]:
# Load transaction data for all chains
chain_data = {}
chain_dfs = {}

for chain in chains:
    print(f'=== LOADING {chain.upper()} DATA ===')
    with open(f'data/validator_announce/{chain}_txs.json', 'r') as f:
        chain_data[chain] = json.load(f)
    
    print(f'Total transactions in dataset: {len(chain_data[chain]["items"])}')
    
    # Convert to DataFrame
    chain_dfs[chain] = pd.DataFrame(chain_data[chain]['items'])
    
    # Display basic info
    print(f'DataFrame shape: {chain_dfs[chain].shape}')
    print(f'Columns: {list(chain_dfs[chain].columns)}')
    print()

# Create individual variables for backward compatibility
sepolia_data = chain_data['sepolia']
sepolia_df = chain_dfs['sepolia']
pruv_data = chain_data['pruv']
pruv_df = chain_dfs['pruv']
arbitrumsepolia_data = chain_data['arbitrumsepolia']
arbitrumsepolia_df = chain_dfs['arbitrumsepolia']

=== LOADING ARBITRUMSEPOLIA DATA ===
Total transactions in dataset: 4
DataFrame shape: (4, 36)
Columns: ['priority_fee', 'raw_input', 'result', 'hash', 'max_fee_per_gas', 'revert_reason', 'confirmation_duration', 'transaction_burnt_fee', 'type', 'token_transfers_overflow', 'confirmations', 'position', 'max_priority_fee_per_gas', 'transaction_tag', 'created_contract', 'value', 'from', 'gas_used', 'status', 'to', 'authorization_list', 'method', 'fee', 'actions', 'gas_limit', 'gas_price', 'decoded_input', 'token_transfers', 'base_fee_per_gas', 'timestamp', 'nonce', 'historic_exchange_rate', 'transaction_types', 'exchange_rate', 'block_number', 'has_error_in_internal_transactions']

=== LOADING PRUV DATA ===
Total transactions in dataset: 7
DataFrame shape: (7, 36)
Columns: ['priority_fee', 'raw_input', 'result', 'hash', 'max_fee_per_gas', 'revert_reason', 'confirmation_duration', 'transaction_burnt_fee', 'type', 'token_transfers_overflow', 'confirmations', 'position', 'max_priority_fee_pe

## 2. Filter and Process Announce Transactions

In [3]:
# Process transactions for all chains
announce_txs = {}

for chain in chains:
    print(f'=== {chain.upper()} ANNOUNCE TRANSACTION FILTERING ===')
    
    # Get the dataframe for this chain
    df = chain_dfs[chain].copy()
    
    # Filter only announce method transactions
    announce_txs[chain] = df[df['method'] == 'announce'].copy()
    
    if len(announce_txs[chain]) == 0:
        print('No announce transactions found')
        continue
    
    # Convert timestamp to datetime
    announce_txs[chain]['datetime'] = pd.to_datetime(announce_txs[chain]['timestamp'])
    
    # Convert numeric fields
    announce_txs[chain]['gas_used'] = pd.to_numeric(announce_txs[chain]['gas_used'])
    announce_txs[chain]['gas_limit'] = pd.to_numeric(announce_txs[chain]['gas_limit'])
    announce_txs[chain]['gas_price'] = pd.to_numeric(announce_txs[chain]['gas_price'])
    
    # Calculate gas efficiency
    announce_txs[chain]['gas_efficiency'] = (announce_txs[chain]['gas_used'] / announce_txs[chain]['gas_limit']) * 100
    
    # Calculate transaction fee in ETH
    announce_txs[chain]['tx_fee_wei'] = announce_txs[chain]['gas_used'] * announce_txs[chain]['gas_price']
    announce_txs[chain]['tx_fee_eth'] = announce_txs[chain]['tx_fee_wei'] / 1e18
    
    # Print statistics
    print(f'Announce transactions: {len(announce_txs[chain])}')
    
    if len(announce_txs[chain]) > 0:
        print(f'First transaction: {announce_txs[chain]["datetime"].min()}')
        print(f'Last transaction: {announce_txs[chain]["datetime"].max()}')
    
    print()

# Create individual variables for backward compatibility
sepolia_announce_txs = announce_txs['sepolia']
pruv_announce_txs = announce_txs['pruv']
arbitrumsepolia_announce_txs = announce_txs['arbitrumsepolia']

=== ARBITRUMSEPOLIA ANNOUNCE TRANSACTION FILTERING ===
Announce transactions: 3
First transaction: 2025-07-24 04:49:30+00:00
Last transaction: 2025-07-24 04:49:31+00:00

=== PRUV ANNOUNCE TRANSACTION FILTERING ===
Announce transactions: 6
First transaction: 2025-07-24 04:41:57+00:00
Last transaction: 2025-08-07 02:19:05+00:00

=== SEPOLIA ANNOUNCE TRANSACTION FILTERING ===
Announce transactions: 3
First transaction: 2025-07-24 04:47:36+00:00
Last transaction: 2025-07-24 04:47:36+00:00



## 3. Detailed Transaction Tables

In [4]:
# Display detailed transaction tables for each chain
for chain in chains:
    if len(announce_txs[chain]) == 0:
        continue
        
    print(f'=== {chain.upper()} DETAILED ANNOUNCE TRANSACTION TABLE ===')
    print()
    
    # Create display table
    display_df = announce_txs[chain][['datetime', 'hash', 'gas_limit', 'gas_used', 'gas_price', 'gas_efficiency', 'tx_fee_eth']].copy()
    display_df.columns = ['Timestamp', 'TX Hash', 'Gas Limit', 'Gas Used', 'Gas Price (Wei)', 'Gas Efficiency (%)', 'TX Fee (ETH)']
    
    # Format columns
    display_df['Gas Efficiency (%)'] = display_df['Gas Efficiency (%)'].round(2)
    display_df['TX Fee (ETH)'] = display_df['TX Fee (ETH)'].round(8)
    
    # Sort by timestamp (newest first)
    display_df = display_df.sort_values('Timestamp', ascending=False)
    
    display(display_df)
    print()

=== ARBITRUMSEPOLIA DETAILED ANNOUNCE TRANSACTION TABLE ===



,Timestamp,TX Hash,Gas Limit,Gas Used,Gas Price (Wei),Gas Efficiency (%),TX Fee (ETH)
0,2025-07-24 04:49:31+00:00,0x15aeb03115c19b7019eb9ed95bc930f287a0687e0932...,335510,216705,100000000,64.59,0.000022
1,2025-07-24 04:49:30+00:00,0x24ba2c8442b2ab6b18376ea2d741d161898a872cbb7b...,335510,216705,100000000,64.59,0.000022
2,2025-07-24 04:49:30+00:00,0xcf656d83feff043605f8de69d4fd3f3a4c244936efeb...,335510,233805,100000000,69.69,0.000023



=== PRUV DETAILED ANNOUNCE TRANSACTION TABLE ===



,Timestamp,TX Hash,Gas Limit,Gas Used,Gas Price (Wei),Gas Efficiency (%),TX Fee (ETH)
0,2025-08-07 02:19:05+00:00,0x8749b002e2a2556baf8291cc0cfee6761cda00f3f39e...,204195,129195,0,63.27,0.0
1,2025-08-07 02:18:45+00:00,0x1dc3112af495b101176492f9dde7e4f251bc89c33c4e...,204195,129195,0,63.27,0.0
2,2025-08-07 02:18:39+00:00,0xb87ff3c9fbe8aef9f3b2dd782db05887e95259181d63...,204171,129171,0,63.27,0.0
3,2025-07-24 04:41:57+00:00,0xef20496e6343a9aff72e685d3d068842db6d745949c2...,308745,216645,0,70.17,0.0
4,2025-07-24 04:41:57+00:00,0x19b2f41385166605cbd1f90c6b1844f4562770788628...,308757,216657,0,70.17,0.0
5,2025-07-24 04:41:57+00:00,0x241d517633e0410a121328f14ee5ced4ce726534627a...,308745,233745,0,75.71,0.0



=== SEPOLIA DETAILED ANNOUNCE TRANSACTION TABLE ===



,Timestamp,TX Hash,Gas Limit,Gas Used,Gas Price (Wei),Gas Efficiency (%),TX Fee (ETH)
0,2025-07-24 04:47:36+00:00,0x8bed7bdd42c969522050df90d5619f1c769e88013a60...,288897,193955,110001666,67.14,0.000021
1,2025-07-24 04:47:36+00:00,0x9e3d19eb6033681940a98e305279b699f451c6e1a00d...,288897,193955,110001666,67.14,0.000021
2,2025-07-24 04:47:36+00:00,0xbf1097acaec5b7ec6044d2f53168b7652540772397b5...,288897,211055,110001666,73.06,0.000023


## 4. Gas Usage Summary

In [5]:
print('=== VALIDATOR ANNOUNCE GAS USAGE SUMMARY ===')
print()

summary_data = []

for chain in chains:
    if len(announce_txs[chain]) == 0:
        continue
        
    txs = announce_txs[chain]
    
    avg_gas_limit = txs['gas_limit'].mean()
    avg_gas_used = txs['gas_used'].mean()
    avg_gas_efficiency = txs['gas_efficiency'].mean()
    avg_gas_price = txs['gas_price'].mean()
    avg_tx_fee_eth = txs['tx_fee_eth'].mean()
    total_transactions = len(txs)
    
    summary_data.append({
        'Chain': chain.title(),
        'Total Announce Calls': total_transactions,
        'Avg Gas Limit': f'{avg_gas_limit:,.0f}',
        'Avg Gas Used': f'{avg_gas_used:,.0f}',
        'Avg Gas Efficiency (%)': f'{avg_gas_efficiency:.2f}%',
        'Avg Gas Price (Wei)': f'{avg_gas_price:,.0f}',
        'Avg TX Fee (ETH)': f'{avg_tx_fee_eth:.8f}'
    })

summary_df = pd.DataFrame(summary_data)
display(summary_df)

=== VALIDATOR ANNOUNCE GAS USAGE SUMMARY ===



,Chain,Total Announce Calls,Avg Gas Limit,Avg Gas Used,Avg Gas Efficiency (%),Avg Gas Price (Wei),Avg TX Fee (ETH)
0,Arbitrumsepolia,3,"335,510","222,405",66.29%,"100,000,000",0.00002224
1,Pruv,6,"256,468","175,768",67.64%,0,0.00000000
2,Sepolia,3,"288,897","199,655",69.11%,"110,001,666",0.00002196
